In [ ]:
!pip uninstall -y jax jaxlib
!pip install -Uqq datasets optuna kaleido

from google.colab import userdata, runtime
import subprocess

hf_token = userdata.get('hf_token')
input_str = f'{hf_token}\nn\n'
result = subprocess.run(['huggingface-cli', 'login'], input=input_str, text=True, capture_output=True)
print(result.stdout)

Found existing installation: jax 0.4.26
Uninstalling jax-0.4.26:
  Successfully uninstalled jax-0.4.26
Found existing installation: jaxlib 0.4.26+cuda12.cudnn89
Uninstalling jaxlib-0.4.26+cuda12.cudnn89:
  Successfully uninstalled jaxlib-0.4.26+cuda12.cudnn89
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.7 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _

In [ ]:
from functools import reduce
import os
import random
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, default_collate
from torchvision.transforms import Resize, Normalize, ToTensor, Compose, transforms, CenterCrop, RandomCrop, RandomChoice
from torchvision.transforms.v2 import CutMix
from sklearn.metrics import f1_score
import optuna
import pickle
import json
import colorsys
import math

from datasets import load_dataset, concatenate_datasets

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

focus = 'augs'
results_dir = f'/content/drive/MyDrive/Colab_Notebooks/dye_test_opt/ternary/results/{focus}'
lr_study_path = results_dir.replace(f'{focus}', 'learning/study.pkl')
synthetic_study_path = results_dir.replace(f'{focus}', 'synthetic_properties/study.pkl')

with open(lr_study_path, 'rb') as f:
    learning_study = pickle.load(f)

with open(synthetic_study_path, 'rb') as f:
    synthetic_study = pickle.load(f)

# Get the best parameters
learning_params = learning_study.best_params
synthetic_study_params = synthetic_study.best_params

# Extract the learning rate and batch size
lr = learning_params.get('lr')
bs = learning_params.get('bs')

n_trials = 150

# Load dataset
ds = load_dataset('mpg-ranch/dye_test', split='train')

# Dataset preparation
total_samples = len(ds)
examples_per_set = 88 #multiplied by 3x months later
n_workers = os.cpu_count()
print(f'Number of available CPU cores: {n_workers}')
n_epochs = 10
n_seeds = 3
context_sz = 154
canvas_sz = context_sz + 14*6 # 1x1 meter context

imagenet_stats = {'mean': [0.485, 0.456, 0.406],
                  'std': [0.229, 0.224, 0.225]}

# Preprocessing transforms
preprocs = Compose([
    CenterCrop((canvas_sz, canvas_sz))
])

def preproc_transforms(examples):
    examples["img"] = [preprocs(image.convert("RGB")) for image in examples["image"]]
    return examples

print("Applying preprocessing transforms...")
ds = ds.map(preproc_transforms, remove_columns=["image","color","size","concentration"], batched=True, batch_size=len(ds))
n_classes = len(np.unique(ds['label']))

def filter_month(batch, month):
    """Returns a list of booleans indicating whether each example in the batch should be kept."""
    # Assuming 'month' is a key in your dataset's dictionary and it's stored in a list for each batch
    return [example_month == month for example_month in batch['month']]

# Define model loading function
def load_model(arch, n_classes):
    print("Loading model...")
    model = torch.hub.load('facebookresearch/dinov2', arch)
    num_ftrs = model.norm.normalized_shape[0]
    model.head = nn.Linear(num_ftrs, n_classes)
    model.to(device)
    return model


def modified_f1_score(labels, predictions):
    scores = []

    # For 1 vs. 0, exclude class 2 and consider only 1 and 0 for binary comparison
    mask_1_vs_0 = (labels != 2)
    labels_1_vs_0 = labels[mask_1_vs_0] == 1
    predictions_1_vs_0 = predictions[mask_1_vs_0] == 1
    if np.any(labels_1_vs_0) or np.any(predictions_1_vs_0):
        f1_1_vs_0 = f1_score(labels_1_vs_0, predictions_1_vs_0, pos_label=True, average='binary')
        scores.append(f1_1_vs_0)

    # For 2 vs. 0, exclude class 1 and consider only 2 and 0 for binary comparison
    mask_2_vs_0 = (labels != 1)
    labels_2_vs_0 = labels[mask_2_vs_0] == 2
    predictions_2_vs_0 = predictions[mask_2_vs_0] == 2
    if np.any(labels_2_vs_0) or np.any(predictions_2_vs_0):
        f1_2_vs_0 = f1_score(labels_2_vs_0, predictions_2_vs_0, pos_label=True, average='binary')
        scores.append(f1_2_vs_0)

    # Calculate the mean of the F1 scores if any valid scores were calculated
    mean_f1 = np.mean(scores) if scores else 0.0

    return f1_1_vs_0, f1_2_vs_0, mean_f1

class SuperimposeSquare(object):
    def __init__(self, red_hue=0.83, blue_hue=0.45,
                 red_value=0.4, blue_value=0.4,
                 red_saturation=0.4, blue_saturation=0.4,
                 max_opacity=0.3, min_opacity=0.1):
        self.red_hue = red_hue
        self.blue_hue = blue_hue
        self.red_value = red_value
        self.blue_value = blue_value
        self.red_saturation = red_saturation
        self.blue_saturation = blue_saturation
        self.max_opacity = max(0, min(1, max_opacity))
        self.min_opacity = max(0, min(1, min_opacity))

    def __call__(self, tensor):
      image = tensor.unsqueeze(0)
      h, w = image.size()[-2:]

      # Randomly choose between small and large box sizes
      small_box = random.choice([True, False])
      if small_box:
          mask_size = 15
      else:
          mask_size = 77

      color_choice = random.choice(['blue', 'red'])
      if color_choice == 'red':
          hue = self.red_hue
          value = self.red_value
          saturation = self.red_saturation
          label = 2
      else:
          hue = self.blue_hue
          value = self.blue_value
          saturation = self.blue_saturation
          label = 1

      saturation = 1.0  # Full saturation for vivid colors
      color_rgb = colorsys.hsv_to_rgb(hue, saturation, value)
      color_tensor = torch.tensor(color_rgb)

      x = (w - mask_size) // 2  # Centering the square on x-axis
      y = (h - mask_size) // 2  # Centering the square on y-axis

      square = color_tensor.view(3, 1, 1).expand(-1, mask_size, mask_size)
      opacity = random.uniform(self.min_opacity, self.max_opacity)
      square = opacity * square + (1 - opacity) * image[:, :, y:y+mask_size, x:x+mask_size]
      image[:, :, y:y+mask_size, x:x+mask_size] = square

      return image.squeeze(0), label

# Define Optuna objective function
def objective(trial):
    print(f"Starting trial: {trial.number}")

    # Hyperparameters to tune
    synthetic_prob = trial.suggest_float('synthetic_prob', 0, 1, step=0.01)
    random_crop_prob = trial.suggest_float('random_crop_prob', 0, 1, step=0.01)
    random_horizontal_flip = trial.suggest_categorical('random_horizontal_flip', [True, False])
    random_vertical_flip = trial.suggest_categorical('random_vertical_flip', [True, False])
    random_rotation = trial.suggest_int('random_rotation', 0, 90, step=5)
    brightness = trial.suggest_float('brightness', 0, 1, step=0.01)
    contrast = trial.suggest_float('contrast', 0, 1, step=0.01)
    saturation = trial.suggest_float('saturation', 0, 1, step=0.01)
    hue = trial.suggest_float('hue', 0, 0.5, step=0.01)

    datasets_by_month = {
        month: ds.filter(function=filter_month, batched=True, batch_size=len(ds), fn_kwargs={'month': month})
        for month in ['may', 'july', 'october']
    }

    # Train transforms
    train_tfms = Compose([
        RandomChoice([RandomCrop(size=context_sz), CenterCrop(context_sz)], p=[random_crop_prob, 1 - random_crop_prob]),
        transforms.RandomHorizontalFlip(p=0.5 if random_horizontal_flip else 0),
        transforms.RandomVerticalFlip(p=0.5 if random_vertical_flip else 0),
        transforms.RandomRotation(random_rotation),
        transforms.ColorJitter(brightness=brightness, contrast=contrast, saturation=saturation, hue=hue),
        #ToTensor applied in collate_fn
        Normalize(mean=imagenet_stats['mean'], std=imagenet_stats['std'])  # Normalize using ImageNet stats
    ])

    val_tfms = Compose([
        CenterCrop(size=context_sz),
        ToTensor(),
        Normalize(mean=imagenet_stats['mean'], std=imagenet_stats['std'])  # Normalize using ImageNet stats
    ])

    def batch_tfms_val(examples):
        examples["img"] = [val_tfms(image) for image in examples["img"]]
        return examples

    f1_scores = {epoch: [] for epoch in range(1, n_epochs + 1)}

    # Seed loop
    for seed in range(n_seeds):
        print(f"Seed: {seed}")
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

        train_ds_list = []
        val_ds_list = []

        # Sample examples_per_set for training and validation from each month's dataset
        for month, dataset in datasets_by_month.items():
            total_samples = len(dataset)
            all_indices = list(range(total_samples))
            sampled_indices = random.sample(all_indices, 2 * examples_per_set)
            train_indices = sampled_indices[:examples_per_set]
            val_indices = sampled_indices[examples_per_set:examples_per_set*2]

            train_ds_month = dataset.select(train_indices)
            val_ds_month = dataset.select(val_indices)

            train_ds_list.append(train_ds_month)
            val_ds_list.append(val_ds_month)

        # Combine the training and validation sets across months
        train_ds_combined = reduce(lambda x, y: concatenate_datasets([x, y]), train_ds_list)
        val_ds_combined = reduce(lambda x, y: concatenate_datasets([x, y]), val_ds_list)

        val_ds_combined.set_transform(batch_tfms_val)

        # Model, optimizer, and loss function setup
        model = load_model('dinov2_vitb14', n_classes)
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
        criterion = nn.CrossEntropyLoss().to(device)

        def collate_fun(batch):
          imgs = [ToTensor()(item['img']) for item in batch]  # Convert PIL images to tensors
          imgs = torch.stack(imgs)
          labels = torch.tensor([item['label'] for item in batch])

          new_imgs = []
          new_labels = []

          for img, label in zip(imgs, labels):
              if label == 0 and synthetic_prob > random.random():
                  transformed_img, new_label = SuperimposeSquare(red_hue=synthetic_study_params.get('red_hue'), blue_hue=synthetic_study_params.get('blue_hue'),
                                                                  red_value=synthetic_study_params.get('red_value'), blue_value=synthetic_study_params.get('blue_value'),
                                                                  red_saturation=synthetic_study_params.get('red_saturation'), blue_saturation=synthetic_study_params.get('blue_saturation'),
                                                                  max_opacity=synthetic_study_params.get('max_opacity'), min_opacity=synthetic_study_params.get('min_opacity'))(img)
                  transformed_img = train_tfms(transformed_img)
                  new_imgs.append(transformed_img)
                  new_labels.append(new_label)
              else:
                  img = train_tfms(img)
                  new_imgs.append(img)
                  new_labels.append(label)

          imgs = torch.stack(new_imgs)
          labels = torch.tensor(new_labels)
          # Wrap the result in a dictionary
          return {'img': imgs, 'label': labels}

        train_loader = DataLoader(train_ds_combined, batch_size=bs, shuffle=True, num_workers=n_workers, collate_fn=collate_fun)
        val_loader = DataLoader(val_ds_combined, batch_size=bs, num_workers=n_workers)

        # Training loop
        for epoch in range(1, n_epochs +1):
            print(f"Epoch: {epoch}/{n_epochs}")
            model.train()
            for _, data in enumerate(train_loader):
                inputs, labels = data['img'].to(device), data['label'].to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            # Validation loop
            model.eval()
            all_labels, all_predictions = [], []
            with torch.no_grad():
                for data in val_loader:
                    inputs, labels = data['img'].to(device), data['label'].to(device)
                    outputs = model(inputs)
                    _, predicted = torch.max(outputs.data, 1)
                    all_labels.extend(labels.cpu().numpy())
                    all_predictions.extend(predicted.cpu().numpy())

            # Compute F1 score
            blue, red, f1 = modified_f1_score(np.array(all_labels), np.array(all_predictions))
            print(f"Blue F1 score: {blue:.4f}\nRed F1 score: {red:.4f}\nMean F1 score: {f1:.4f}")

            f1_scores[epoch].append(f1)

    # Return the mean F1 score over the seeds
    mean_f1_scores = {epoch: np.mean(scores) for epoch, scores in f1_scores.items()}
    highest_f1_epoch, highest_f1_score = max(mean_f1_scores.items(), key=lambda x: x[1])

    print(f"Highest F1-score {highest_f1_score} at epoch {highest_f1_epoch}")

    return highest_f1_score

def study_cb(study, trial):
    patience = 50

    # Directory where the figures and trial data will be saved
    os.makedirs(results_dir, exist_ok=True)

    # Save figures
    if trial.state == optuna.trial.TrialState.COMPLETE:
        opt_hist_path = os.path.join(results_dir, f'opt_history_plot.png')
        fig1 = optuna.visualization.plot_optimization_history(study)
        fig1.write_image(opt_hist_path)

        opt_slice_path = os.path.join(results_dir, f'opt_slice_plot.png')
        fig2 = optuna.visualization.plot_slice(study)
        fig2.write_image(opt_slice_path)

    study_path = os.path.join(results_dir, 'study.pkl')
    with open(study_path, 'wb') as f:
        pickle.dump(study, f)

    if 'best_trial_number' not in study.user_attrs:
        # Initialize with the first trial
        study.set_user_attr('best_trial_number', 0)

    current_best_trial = study.best_trial.number
    best_trial_number = study.user_attrs['best_trial_number']

    # Update the best trial number in study attributes if the current best is newer
    if current_best_trial > best_trial_number:
        study.set_user_attr('best_trial_number', current_best_trial)
        best_trial_number = current_best_trial

    # Check if no improvement has been made for 'tolerance' trials
    if trial.number >= best_trial_number + patience:
        study.stop()


study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=0))
study.optimize(objective, n_trials=n_trials, callbacks=[study_cb])

runtime.unassign()

Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Number of available CPU cores: 8
Applying preprocessing transforms...


...

Epoch: 1/10
Blue F1 score: 0.4494
Red F1 score: 0.3256
Mean F1 score: 0.3875
Epoch: 2/10
Blue F1 score: 0.1509
Red F1 score: 0.7000
Mean F1 score: 0.4255
Epoch: 3/10
Blue F1 score: 0.6947
Red F1 score: 0.7344
Mean F1 score: 0.7146
Epoch: 4/10
Blue F1 score: 0.6087
Red F1 score: 0.8148
Mean F1 score: 0.7118
Epoch: 5/10
Blue F1 score: 0.7865
Red F1 score: 0.8727
Mean F1 score: 0.8296
Epoch: 6/10
Blue F1 score: 0.8095
Red F1 score: 0.8889
Mean F1 score: 0.8492
Epoch: 7/10
Blue F1 score: 0.8409
Red F1 score: 0.9216
Mean F1 score: 0.8812
Epoch: 8/10
Blue F1 score: 0.8750
Red F1 score: 0.8718
Mean F1 score: 0.8734
Epoch: 9/10
Blue F1 score: 0.8989
Red F1 score: 0.9107
Mean F1 score: 0.9048
Epoch: 10/10
Blue F1 score: 0.7928
Red F1 score: 0.9524
Mean F1 score: 0.8726
Seed: 1
Loading model...


Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main


Epoch: 1/10
Blue F1 score: 0.2817
Red F1 score: 0.2727
Mean F1 score: 0.2772
Epoch: 2/10
Blue F1 score: 0.2769
Red F1 score: 0.5926
Mean F1 score: 0.4348
Epoch: 3/10
Blue F1 score: 0.5495
Red F1 score: 0.7302
Mean F1 score: 0.6398
Epoch: 4/10
Blue F1 score: 0.5000
Red F1 score: 0.6667
Mean F1 score: 0.5833
Epoch: 5/10
Blue F1 score: 0.7073
Red F1 score: 0.9459
Mean F1 score: 0.8266
Epoch: 6/10
Blue F1 score: 0.7115
Red F1 score: 0.9459
Mean F1 score: 0.8287
Epoch: 7/10
Blue F1 score: 0.8235
Red F1 score: 0.9333
Mean F1 score: 0.8784
Epoch: 8/10
Blue F1 score: 0.8667
Red F1 score: 0.9231
Mean F1 score: 0.8949
Epoch: 9/10
Blue F1 score: 0.8864
Red F1 score: 0.9474
Mean F1 score: 0.9169
Epoch: 10/10
Blue F1 score: 0.8605
Red F1 score: 0.9730
Mean F1 score: 0.9167
Seed: 2
Loading model...


Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main


Epoch: 1/10
Blue F1 score: 0.3962
Red F1 score: 0.3448
Mean F1 score: 0.3705
Epoch: 2/10
Blue F1 score: 0.5075
Red F1 score: 0.3385
Mean F1 score: 0.4230
Epoch: 3/10
Blue F1 score: 0.5512
Red F1 score: 0.6111
Mean F1 score: 0.5811
Epoch: 4/10
Blue F1 score: 0.5773
Red F1 score: 0.7838
Mean F1 score: 0.6806
Epoch: 5/10
Blue F1 score: 0.6598
Red F1 score: 0.8571
Mean F1 score: 0.7585
Epoch: 6/10
Blue F1 score: 0.7568
Red F1 score: 0.8049
Mean F1 score: 0.7808
Epoch: 7/10
Blue F1 score: 0.8158
Red F1 score: 0.8421
Mean F1 score: 0.8289
Epoch: 8/10
Blue F1 score: 0.8571
Red F1 score: 0.8571
Mean F1 score: 0.8571
Epoch: 9/10
Blue F1 score: 0.8642
Red F1 score: 0.8000
Mean F1 score: 0.8321
Epoch: 10/10
